In [ ]:
import pandas as pd
import os
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from src.constants import *
import folium
import IPython
from folium.plugins import HeatMap
import glob
from folium.plugins import HeatMapWithTime
from jinja2 import Template
from folium.map import Layer
from tqdm.auto import tqdm
import joblib
import seaborn as sns
data_twitter = joblib.load(os.path.join(DATA_PATH,"TWITTER_DATA.joblib"))
data_covid = joblib.load(os.path.join(DATA_PATH,"COVID_DATA.joblib"))
data_media = joblib.load(os.path.join(DATA_PATH,"MEDIA_DATA.joblib"))
def generateBaseMap(default_location=[46.900, 8.4375], default_zoom_start=5):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start, tiles ="https://api.mapbox.com/styles/v1/tigerflystudio/ckf9pjhvp1naa19rwl43ahilz/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidGlnZXJmbHlzdHVkaW8iLCJhIjoiY2tmOW94a3kzMG83ZTJybWFqc28ydDI1NyJ9.uoRh84Gv3y9J397hMz-BDw",
           attr='Mapbox Control Room')
    return base_map

In [ ]:
from IPython.display import HTML
HTML('''
<style>
* {
  box-sizing: border-box;
}

/* Create three equal columns that floats next to each other */
.column {
  float: left;
  width: 30%;
  padding: 10px;
}

/* Create three equal columns that floats next to each other */
.column2 {
  text-align: center;
  float: left;
  width: 40%;
  padding: 10px;
}

/* Clear floats after the columns */
.row:after {
  content: "";
  display: table;
  clear: both;
}

/* Responsive layout - makes the three columns stack on top of each other instead of next to each other */
@media screen and (max-width: 600px) {
  .column {
    width: 100%;
  }
}
</style>
</head>
<div class="row">
  <div class="column">
    <img id="plotly-image" src="logo.png" style="height: 70px; width: auto; margin-bottom: 25px;">
  </div>
  <div class="column2">
    <h1 style="font-size:3em;">COVID-19 Scare Level</h1>
<h4 style="margin-top: 0px;">An interactive dashboard for journalists</h4>
  </div>
  <div class="column">
    
  </div>
</div>
''')

In [ ]:
def centering(s):
    return HTML('''
    <style>
    .column2 {
      text-align: center;
      float: left;
      width: 100%;
      padding: 10px;
    }
    .row:after {
      content: "";
      display: table;
      clear: both;
    }

    /* Responsive layout - makes the three columns stack on top of each other instead of next to each other */
    @media screen and (max-width: 600px) {
      .column {
        width: 100%;
      }
    }
    </style>
    </head>
    <div class="row">
      <div class="column2">
        <h2 style="font-size:2em;">''' + s + '''</h2>
      </div>
      <div class="column">

      </div>
    </div>
    ''')

In [ ]:
centering("Corona Mentions in Swiss News Sources over Time")

In [ ]:
base_map = generateBaseMap(default_zoom_start=8)
hmap_time_media = HeatMapWithTime(data_media,overlay=True,name='Media',min_opacity=0.05,radius=20,max_opacity=1, gradient ={0.2: 'lawngreen', 0.5: 'limegreen', 0.7: 'green', 1.0: 'darkgreen'}).add_to(base_map)
base_map.add_child(hmap_time_media)

In [ ]:
centering("Corona Case Data")

In [ ]:
base_map = generateBaseMap(default_zoom_start=1.5)
hmap_time_media = HeatMapWithTime(data_covid,overlay=True,name='COVID',min_opacity=0.05,radius=20,max_opacity=1, gradient ={0.2: 'yellow', 0.5: 'gold', 0.8: 'orange', 1.0: 'red'}).add_to(base_map)
base_map.add_child(hmap_time_media)

In [ ]:
centering("Corona Related Tweets")

In [ ]:
base_map = generateBaseMap(default_zoom_start=6)
hmap_time_media = HeatMapWithTime(data_twitter,overlay=True,name='Twitter',min_opacity=0.05,radius=10,max_opacity=1, gradient ={0.2: 'lightsteelblue', 0.5: 'cornflowerblue', 0.7: 'royalblue', 1.0: 'navy'}).add_to(base_map)
base_map.add_child(hmap_time_media)

In [ ]:
centering("Corona Scare Level in Switzerland")

In [ ]:
#LOAD AND CLEAN COVID DATA

scare_twitter = joblib.load(os.path.join(DATA_PATH,"Twitter_Scare_Score.joblib")).copy()
scare_media = pd.read_csv(os.path.join(DATA_PATH,"srf_data/processed/Swiss_media_score.csv"))
scare_media.pubDateTime = pd.to_datetime(scare_media.pubDateTime)
scare_media = scare_media[(scare_media["pubDateTime"] > "2020-01-20") & (scare_media["pubDateTime"] < "2020-06-08")]
scare_sentiment = pd.read_csv(os.path.join(DATA_PATH,"srf_data/processed/Swiss_media_corona_sentiment_score.csv"))
scare_sentiment.pubDateTime = pd.to_datetime(scare_sentiment.pubDateTime)
scare_sentiment = scare_sentiment[(scare_sentiment["pubDateTime"] > "2020-01-20") & (scare_sentiment["pubDateTime"] < "2020-06-08")]
scare_sentiment= scare_sentiment.set_index("pubDateTime")

plot_data = scare_media[["pubDateTime", "Media_Score"]]
plot_data= plot_data.set_index("pubDateTime")

plot_data = plot_data.rename(columns={"Media_Score": "Media"})
scare_sentiment = scare_sentiment.rename(columns={"Score": "Sentiment"})
scare_twitter = scare_twitter.rename(columns={"count": "Twitter"})

#ax = sns.lineplot(data =scare_twitter, palette=['blue'])
#sns.lineplot(data = plot_data, ax=ax, palette=['green'])

In [61]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show

def datetime(x):
    return np.array(x, dtype=np.datetime64)

p1 = figure(x_axis_type="datetime", title="Corona Scare")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Scare'
p1.yaxis.axis_label = 'Time'

p1.line(scare_twitter.index, scare_twitter.Twitter, color='red', legend_label='Twitter')
p1.line(plot_data.index, plot_data.Media, color='green', legend_label='Media')
p1.legend.location = "top_left"

p2 = figure(x_axis_type="datetime", title="Sentiment of Corona Related Articles")
p2.grid.grid_line_alpha=0.3
p2.xaxis.axis_label = 'Sentiment'
p2.yaxis.axis_label = 'Time'

p2.line(scare_sentiment.index, scare_sentiment.Sentiment, color='blue', legend_label='Sentiment')
output_notebook(hide_banner=True)


In [58]:
show(gridplot([[p1,p2]], plot_width=680, plot_height=600))  # open a browser